# Linear Regression with Regularization

## Problem Statement

Predict the bike-sharing counts per hour based on the features including weather, day, time, humidity, wind speed, season e.t.c.

## Objectives

* perform data exploration and visualization
* implement linear regression using sklearn and optimization
* apply regularization on regression using Lasso, Ridge and Elasticnet techniques
* calculate and compare the MSE value of each regression technique
* analyze the features that are best contributing to the target

### Dataset

The dataset chosen for this project is [Bike Sharing Dataset](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset).  This dataset contains the hourly and daily count of rental bikes between the years 2011 and 2012 in the capital bike share system with the corresponding weather and seasonal information. This dataset consists of 17389 instances of 16 features. 

### Data Fields

* dteday - hourly date
* season - 1 = spring, 2 = summer, 3 = fall, 4 = winter
* hr - hour
* holiday - whether the day is considered a holiday
* workingday - whether the day is neither a weekend nor holiday
* weathersit -<br>
    1 - Clear, Few clouds, Partly cloudy, Partly cloudy <br>
    2 - Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist<br>
    3 - Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds<br>
    4 - Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog<br>   
* temp - temperature in Celsius
* atemp - "feels like" temperature in Celsius
* humidity - relative humidity
* windspeed - wind speed
* casual - number of non-registered user rentals initiated
* registered - number of registered user rentals initiated
* cnt - number of total rentals

## Download the dataset

#### Importing Necessary Packages

In [ ]:
# Loading the Required Packages
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score

### Data Loading

In [ ]:
# Read the hour.csv file
df = pd.read_csv('hour.csv')

print the first five rows of dataset

In [ ]:
df.head(5)

print the datatypes of the columns

In [ ]:
df.dtypes

### Task flow with respect to feature processing and model training

* Explore and analyze the data

* Identify continuous features and categorical features

* Apply scaling on continuous features and one-hot encoding on categorical features

* Separate the features, targets and split the data into train and test

* Find the coefficients of the features using normal equation and find the cost (error)

* Apply batch gradient descent technique and find the best coefficients

* Apply SGD Regressor using sklearn

* Apply linear regression using sklearn

* Apply Lasso, Ridge, Elasticnet Regression

### EDA &  Visualization 

#### Visualize the hour (hr) column and find the busy hours of bike sharing

In [ ]:
plt.bar(df['hr'], df['cnt'])
plt.show()


#### Visualize the distribution of count, casual and registered variables

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.distplot(df['cnt'])

In [ ]:
sns.histplot(df['cnt'].sort_values(ascending=False))

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.distplot(df['casual'])

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.distplot(df['registered'])

#### Describe the relation of weekday, holiday and working day

In [ ]:
df1 = df[['weekday', 'holiday', 'workingday']]
df1.corr()

#### Visualize the month wise count of both casual and registered for the year 2011 and 2012 separately.

In [ ]:
df['dteday'] = pd.to_datetime(df['dteday'])

In [ ]:
df

In [ ]:
temp1 = df[df['yr'] == 0]
temp2 = df[df['yr'] == 1]

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
temp1[['mnth', 'casual', 'registered']].groupby(by='mnth').count().plot(kind='bar', stacked=True, ax=ax)

In [ ]:
# stacked bar chart for year 2012
fig, ax = plt.subplots(figsize=(12,8))
temp2[['mnth', 'casual', 'registered']].groupby(by='mnth').count().plot(kind='bar', stacked=True, ax=ax)

#### Analyze the correlation between features with heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(df.corr(), annot=True)

#### Visualize the box plot of casual and registered variables to check the outliers

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.boxplot(data=df, y='casual', ax=ax)


In [ ]:
fig, ax = plt.subplots(figsize=(10,15))
sns.boxplot(data=df, y='registered', ax=ax)


### Pre-processing and Data Engineering 

#### Drop unwanted columns

In [ ]:
df.head(5)

In [ ]:
df = df.drop(columns=['instant', 'dteday'])

#### Identify categorical and continuous variables


In [ ]:
categorical = []
numerical = []
for cols in df.columns:
  if df[cols].nunique() <= 15:
    categorical.append(cols)
  else:
    numerical.append(cols)

In [ ]:
for col in categorical:
  if col =='season':
    categorical.remove(col)
    numerical.append(col)
  if col =='weathersit':
    categorical.remove(col)
    numerical.append(col)

#### Feature scaling

Apply scaling on the continuous variables on the given data.



In [ ]:
sc = StandardScaler()
df[numerical] = sc.fit_transform(df[numerical])

In [ ]:
min_max = MinMaxScaler()
df[numerical] = min_max.fit_transform(df[numerical])

#### Apply one-hot encode on the categorical data

In [ ]:
ohe = OneHotEncoder()
hour_df_copy = df.copy()
#hour_df_ohe = pd.get_dummies(hour_df_copy, columns=['season', 'mnth', 'hr', 'weathersit'], prefix = ['season', 'mnth', 'hr', 'weathersit'])
hour_df_ohe = pd.get_dummies(df, columns = categorical)
hour_df_ohe.columns

In [ ]:
# encoder = OneHotEncoder(handle_unknown='ignore')
# encoder.fit(df[categorical])
# category_columns_encoded = encoder.fit_transform(df[categorical]).toarray()
# category_cols_df = pd.DataFrame(category_columns_encoded, columns=encoder.get_feature_names(categorical))
# df_encoded = pd.concat([df.drop(columns=categorical), category_cols_df], axis=1)

In [ ]:
hour_df_ohe.head()

#### Specify features and targets after applying scaling and one-hot encoding

In [ ]:
features = hour_df_ohe.drop(columns=['cnt'])
target = hour_df_ohe[['cnt']]

### Implement the linear regression by finding the coefficients using below approaches

* Find the coefficients using normal equation

* (Optional) Implement batch gradient descent 

* (Optional) SGD Regressor from sklearn

#### Select the features and target and split the dataset

As there are 3 target variables, choose the count (`cnt`) variable.

In [ ]:
X = hour_df_ohe[features]
y = hour_df_ohe[target]
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, '\n', X_test.shape, y_test.shape)

#### Implementation using Normal Equation

$\theta = (X^T X)^{-1} . (X^T Y)$

$θ$ is the hypothesis parameter that defines the coefficients

$X$ is the input feature value of each instance

$Y$ is Output value of each instance

In [ ]:
theta = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)
theta

#### Implementing Linear regression using batch gradient descent

Initialize the random coefficients and optimize the coefficients in the iterative process by calculating cost and finding the gradient.

In [ ]:
def cost_function(X, Y, B):
 m = len(Y)
 J = np.sum((X.dot(B) - Y) ** 2)/2*m
 return J

In [ ]:
def batch_gradient_descent(X, Y, B, alpha, iterations):
 cost_history = [0] * iterations
 m = len(Y)
 
 for iteration in range(iterations):
    #print(iteration)
    # Hypothesis Values
    h = X.dot(B)
    # Difference b/w Hypothesis and Actual Y
    loss = h - Y
    # Gradient Calculation
    gradient = X.T.dot(loss) / m
    # Changing Values of B using Gradient
    B = B - (alpha * gradient)
    # New Cost Value
    cost = cost_function(X, Y, B)
    cost_history[iteration] = cost
  
 return B, cost_history

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features.values, target.values, test_size= 0.2, random_state= 0)

In [ ]:
X_train

In [ ]:
X_train = np.c_[np.ones(len(X_train),dtype='int64'),X_train]
X_test = X_test = np.c_[np.ones(len(X_test),dtype='int64'),X_test]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
B = np.zeros(X_train.shape[1])
alpha = 0.1
iter_ = 50
newB, cost_history = batch_gradient_descent(X_train, y_train.ravel(), B, alpha, iter_)

In [ ]:
newB

#### SGD Regressor

* Import SGDRegressor from sklearn and fit the data

* Predict the test data and find the error

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=1000, penalty=None, eta0=0.1)
# Fit the model to the training data
sgd_reg.fit(X_train, y_train.ravel())

In [ ]:
# Make predictions on the test data
y_pred = sgd_reg.predict(X_test)
y_pred

In [ ]:
# Calculate the mean squared error of the predictions
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)

### Linear regression using sklearn

Implement the linear regression model using sklearn

* Import Linear Regression and fit the train data

* Predict the test data and find the error

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
y_pred

#### Calculate the $R^2$ (coefficient of determination) of the actual and predicted data

In [ ]:
r2_score(y_pred, y_test)

#### Summarize the importance of features

Use the coefficients obtained through the sklearn Linear Regression implementation and create a bar chart of the coefficients.

In [ ]:
lr.coef_.shape

In [ ]:
coef_list = []
for coef in lr.coef_[0]:
  coef_list.append(coef)

In [ ]:
np.shape(coef_list)

In [ ]:
lr_coef = lr.coef_

fig, ax = plt.subplots(figsize=(12,9))
plt.bar([x for x in range(len(coef_list))], coef_list)

### Regularization methods

#### Apply Lasso regression

* Apply Lasso regression with different alpha values given below and find the best alpha that gives the least error.
* Calculate the metrics for the actual and predicted

In [ ]:
# setting up alpha
alphas = [0.0001, 0.001,0.01, 0.1, 1, 10, 100]

In [ ]:
for alpha in alphas:
    model = linear_model.Lasso(alpha=alpha)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print("Alpha = {}, R2 = {}".format(alpha, score))

#### Apply Ridge regression

* Apply Ridge regression with different alpha values given and find the best alpha that gives the least error.
* Calculate the metrics for the actual and predicted

In [ ]:
for alpha in alphas:
    model = linear_model.Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print("Alpha = {}, R2 = {}".format(alpha, score))

#### Apply Elasticnet regression

* Apply Elasticnet regression with different alpha values given and find the best alpha that gives the least error.
* Calculate the metrics for the actual and predicted

In [ ]:
for alpha in alphas:
    model = linear_model.ElasticNet(alpha=alpha)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print("Alpha = {}, R2 = {}".format(alpha, score))